# Étape 4 — Jointures (Merge) et enrichissement des ventes

L’objectif de cette étape est d’enrichir les données de ventes (`order_lines`)
avec les informations issues des fichiers clients (`customers`) et produits
(`products`) afin de produire une table d’analyse unique.

Cette étape inclut la vérification des clés de jointure, le contrôle de la
qualité des merges, le recalcul de certaines variables business et une
mini-analyse après enrichissement.

**Import des librairies**

In [ ]:
# Import des librairies nécessaires aux jointures et contrôles de qualité

import pandas as pd
import numpy as np

# Options d'affichage pour une meilleure lisibilité des DataFrames
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

**Chargement des données nettoyées**

In [ ]:
# Chargement des données nettoyées depuis le dossier data/processed

order_lines = pd.read_csv("../data/processed/order_lines_clean.csv")
customers = pd.read_csv("../data/processed/customers_clean.csv")
products = pd.read_csv("../data/processed/products_clean.csv")

# Vérification rapide des dimensions des jeux de données
print("order_lines :", order_lines.shape)
print("customers   :", customers.shape)
print("products    :", products.shape)

# Vérification des clés

**Vérification des clés clients (customer_id)**

In [ ]:
# Vérification de la présence des clés customer_id dans les ventes et les clients

# Nombre de customer_id distincts dans chaque fichier
nb_customers_orders = order_lines["customer_id"].nunique()
nb_customers_customers = customers["customer_id"].nunique()

print(f"Nombre de clients distincts dans order_lines : {nb_customers_orders}")
print(f"Nombre de clients distincts dans customers   : {nb_customers_customers}")

# Identification des customer_id présents dans les ventes mais absents des clients
customers_manquants = set(order_lines["customer_id"]) - set(customers["customer_id"])

print(f"Nombre de customer_id présents dans les ventes mais absents des clients : {len(customers_manquants)}")

**Vérification des clés produits (product_id)**

In [ ]:
# Vérification de la présence des clés product_id dans les ventes et les produits

# Nombre de product_id distincts dans chaque fichier
nb_products_orders = order_lines["product_id"].nunique()
nb_products_products = products["product_id"].nunique()

print(f"Nombre de produits distincts dans order_lines : {nb_products_orders}")
print(f"Nombre de produits distincts dans products    : {nb_products_products}")

# Identification des product_id présents dans les ventes mais absents du catalogue produits
products_manquants = set(order_lines["product_id"]) - set(products["product_id"])

print(f"Nombre de product_id présents dans les ventes mais absents des produits : {len(products_manquants)}")

# Les jointures

**Jointure ventes , clients**

In [ ]:
#Jointure des ventes avec les informations clients
# Clé de jointure : customer_id
# Type de jointure : left 

orders_clients = order_lines.merge(customers,
                                    on= "customer_id",
                                    how= "left",
                                    suffixes=("","_customer")
                                    )

# Vérification du après jointure
orders_clients.shape

**Jointure résultat , produits(product_id, left join)**

In [ ]:
# Jointure du jeu ventes-clients avec les informations produits
# Clé de jointure : product_id
# Type de jointure : left

orders_enriched = orders_clients.merge( products,
                                        on= "product_id",
                                        how= "left",
                                        suffixes=("", "_product")
                                      )

# Vérification de la dimension après jointure
orders_enriched.shape

# Contrôle de la qualité des jointures

**Controle qualité des jointures (taille avant/après)**

In [ ]:
# Taille avant et après jointures

print("Taille orders_lines_clean : ", order_lines.shape)
print("Taille orders_clients : ", orders_clients.shape)
print("Taille orders_enriched:", orders_enriched.shape)

In [ ]:
# Vérification stricte : même nombre de lignes avant/après merge
order_lines.shape[0] == orders_enriched.shape[0]

**Lignes sans correspondance clients**

In [ ]:
missing_client_city = orders_enriched["city"].isna().sum()
missing_client_segment = orders_enriched["segment"].isna().sum()

missing_client_city

**Lignes sans correspondance produit**

In [ ]:
missing_product_category = orders_enriched["category"].isna().sum()
missing_product_price = orders_enriched["unit_price"].isna().sum()

**Tableau résumé**

In [ ]:

control_summary = pd.DataFrame({
    "Problème détecté": [
        "Clients sans ville",
        "Clients sans segment",
        "Produits sans catégorie",
        "Produits sans prix unitaire"
    ],
    "Nombre de lignes": [
        missing_client_city,
        missing_client_segment,
        missing_product_category,
        missing_product_price
    ]
})

control_summary

# Recalcul des colonnes business après enrichissement

**Sécurisation du type de discount_pct**

In [ ]:
orders_enriched["discount_pct"] = pd.to_numeric(
    orders_enriched["discount_pct"],
    errors="coerce"
)

**Recalcul du montant brut**

In [ ]:
orders_enriched["gross_amount_calc"] = (
    orders_enriched["unit_price"] * orders_enriched["quantity"]
)

**Recalcul du montant net**

In [ ]:
orders_enriched["net_amount_calc"] = (
    orders_enriched["gross_amount_calc"] * (1 - orders_enriched["discount_pct"])
)

**Vérification des nouvelles colonnes**

In [ ]:
orders_enriched[
    [
        "unit_price",
        "quantity",
        "discount_pct",
        "gross_amount",
        "gross_amount_calc",
        "net_amount",
        "net_amount_calc"
    ]
].head(40)

# Comparaison des montants et détection des lignes suspectes

**Calcul de l’écart entre montant observé et montant recalculé**

In [ ]:
orders_enriched["amount_diff"] = (
    orders_enriched["net_amount"] - orders_enriched["net_amount_calc"]
)

orders_enriched

**Identification des lignes suspectes**

In [ ]:
# une ligne est suspecte si ∣amount_diff∣>0.01

suspect_orders = orders_enriched[
    orders_enriched["amount_diff"].abs() > 0.01
]


**Nombre de lignes suspectes**

In [ ]:
suspect_orders.shape[0]

# Mini-analyse

**Création du tableau croisé**

In [ ]:
# Vision globale des colonnes du data_set orders_enriched
orders_enriched.columns

In [ ]:
# Création du tableau croisé (Segment en ligne, Catégorie en colonne)
ca_segment_category = pd.pivot_table(
    orders_enriched,
    values="net_amount_calc",
    index="segment_customer",
    columns="category_product",
    aggfunc="sum"
)


In [ ]:
# Calcul du Total par segment (Lignes)
ca_segment_category["Total_CA"] = ca_segment_category.sum(axis=1)

# Calcul du Total par catégorie (Colonnes)
total_row = ca_segment_category.sum(axis=0)
total_row.name = "TOTAL_GLOBAL"



In [ ]:
# Assemblage et tri 
ca_segment_category = pd.concat([
    ca_segment_category.sort_values(by="Total_CA", ascending=False),
    total_row.to_frame().T
])

# Affichage de segment_customer en ligne
ca_segment_category.index.name = "segment_customer"

ca_segment_category


**Export du tableau du chiffre d'affaires par catégorie par segment vers results/tables**

In [ ]:
ca_segment_category.to_csv("../results/tables/ca_segment_categorie.csv", index=False)

**Suppression des colonnes redondantes dans orders_enriched**

In [ ]:
# Suppression des colonnes redondantes
columns_to_drop = ["city", "segment", "category", "unit_price"]
orders_enriched_clean = orders_enriched.drop(columns=columns_to_drop)

# Harmonisation des noms de colonnes
orders_enriched_clean = orders_enriched_clean.rename(columns={
    "city_customer": "city",
    "segment_customer": "segment",
    "category_product": "category",
    "unit_price_product": "unit_price"
})


In [ ]:
orders_enriched_clean.shape

**Export de orders_enriched.csv**

In [ ]:
orders_enriched_clean.to_csv( "../results/tables/orders_enriched.csv", index=False)
print("Export terminé : orders_enriched.csv enregistré dans results/tables/")

# Commentaires